# Lecture 14: ALL-IDB Classification using Autoencoders

### Dataset used:- [ALL-IDB:Acute Lymphoblastic Leukemia Image Database for Image Processing](https://homes.di.unimi.it/scotti/all/)
Follow the instructions provided in the linked website to download the dataset. After downloading, extract the files to the current directory (same folder as your codes).

Load Packages
===============

In [ ]:
%matplotlib inline
import os
import copy
import torch
import random
import numpy as np
import torch.nn as nn
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt

print(torch.__version__) # This code has been updated for PyTorch 1.0.0

Load Data:
===============

In [ ]:
Datapath = 'ALL_IDB2/img/'
listing = os.listdir(Datapath) 
random.shuffle(listing)

In [ ]:
TrainImages = torch.DoubleTensor(200,3072)
TrainLabels = torch.LongTensor(200)
TestImages = torch.DoubleTensor(60,3072)
TestLabels = torch.LongTensor(60)

img_no = 0
for file in listing:
    im=Image.open(Datapath + file)
    im = im.resize((32,32))
    im = np.array(im)
    im = np.reshape(im, 32*32*3)
    if img_no < 200:
        TrainImages[img_no] = torch.from_numpy(im)
        TrainLabels[img_no] = int(listing[img_no][6:7])
    else:
        TestImages[img_no - 200] = torch.from_numpy(im)
        TestLabels[img_no - 200] = int(listing[img_no][6:7])
    img_no = img_no + 1        

In [ ]:
print(TrainImages.size())
print(TrainLabels.size())
print(TestImages.size())
print(TestLabels.size())

In [ ]:
# Checking availability of GPU
use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')
    device = "cuda"
else:
    print('GPU is not available!')
    device = "cpu"

Define the Autoencoder:
===============

In [ ]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(32*32*3, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU())
        self.decoder = nn.Sequential(
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 32*32*3),
            nn.ReLU())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


net = autoencoder()
print(net)

net = net.double().to(device)
        
init_weights = copy.deepcopy(net.encoder[0].weight.data)

Define Optimization Technique:
===============

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.5, momentum=0.9)

Train Autoencoder:
===========

In [ ]:
iterations = 30
BatchSize = 10
for epoch in range(iterations):
    runningLoss = 0
    for i in range(int(TrainImages.size()[0]/BatchSize)):
        inputs = torch.index_select(TrainImages,0,torch.linspace(i*BatchSize,(i+1)*BatchSize - 1,steps=BatchSize)
                                  .long()).double()
        inputs = inputs/255
        inputs = inputs.to(device)
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()
    print('At Iteration : %d / %d  ;  Mean-Squared Error : %f'%(epoch + 1,iterations,runningLoss/
                                                                (TrainImages.size()[0]/BatchSize)))
print('Finished Training')

Encoder Weights Visualization:
=======================

In [ ]:
trained_weights = copy.deepcopy(net.encoder[0].weight.data)
init_weights = (1 + init_weights)*127.5
trained_weights = (1 + trained_weights)*127.5

init_weights = init_weights.view(3,320,320).byte()
trained_weights = trained_weights.view(3,320,320).byte()

if device == 'cuda':
    init_weights = init_weights.cpu()
    trained_weights = trained_weights.cpu()    

d_weights = init_weights - trained_weights 

fig = plt.figure()
plot=fig.add_subplot(1,3,1)
img = np.array(init_weights.numpy())[0]
plot.set_title('Initial Weights')
imgplot = plt.imshow(img)

plot=fig.add_subplot(1,3,2)
img = np.array(trained_weights.numpy())[0]
plot.set_title('Trained Weights')
imgplot = plt.imshow(img)

plot=fig.add_subplot(1,3,3)
img = np.array(d_weights.numpy())[0]
plot.set_title('Weight update')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
new_classifier = nn.Sequential(*list(net.children())[:-1])
net = new_classifier
net.add_module('classifier', nn.Sequential(nn.Linear(100, 2),nn.LogSoftmax(dim=1)))
print(net)
net = net.double().to(device)
cll_weights = copy.deepcopy(net[0][0].weight.data)
init_classifier_weights = copy.deepcopy(net.classifier[0].weight.data)

Define Optimizer:
================================

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

Train Classifier:
===========

In [ ]:
iterations = 30
BatchSize = 10
for epoch in range(iterations):
    runningLoss = 0
    for i in range(int(TrainImages.size()[0]/BatchSize)):
        inputs = torch.index_select(TrainImages,0,torch.linspace(i*BatchSize,(i+1)*BatchSize - 1,steps=BatchSize)
                                  .long()).double()
        labels = torch.index_select(TrainLabels,0,torch.linspace(i*BatchSize,(i+1)*BatchSize - 1,steps=BatchSize)
                                  .long()).long()
        inputs = inputs/255
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()
        
    inputs = TestImages.double()/255
    inputs = inputs.to(device)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    if use_gpu:        
        predicted = predicted.cpu()
    correct = 0
    total = 0
    total += TestLabels.size(0)
    correct += (predicted == TestLabels).sum()
    print('At Iteration: %d / %d  ;  Training Loss: %f ; Testing Acc: %f '%(epoch + 1,iterations,runningLoss/
                                                                            (TrainImages.size()[0]/
                                                                             BatchSize),(100 * correct/ float(total))))
print('Finished Training')

Encoder Weights Visualization:
=======================

In [ ]:
cll_weights_ft = copy.deepcopy(net[0][0].weight.data)
cll_weights = (1 + cll_weights)*127.5
cll_weights_ft = (1 + cll_weights_ft)*127.5

cll_weights = cll_weights.view(3,320,320).byte()
cll_weights_ft = cll_weights_ft.view(3,320,320).byte()
if use_gpu:
    cll_weights = cll_weights.cpu()
    cll_weights_ft = cll_weights_ft.cpu()

d_weights = cll_weights - cll_weights_ft

fig = plt.figure()
plot=fig.add_subplot(1,3,1)
img = np.array(cll_weights.numpy())[0]
plot.set_title('Encoder Weights')
imgplot = plt.imshow(img)

plot=fig.add_subplot(1,3,2)
img = np.array(cll_weights_ft.numpy())[0]
plot.set_title('Finetuned Weights')
imgplot = plt.imshow(img)

plot=fig.add_subplot(1,3,3)
img = np.array(d_weights.numpy())[0]
plot.set_title('Weight update')
imgplot = plt.imshow(img)
plt.show()

Classifier Weights Visualization:
=======================

In [ ]:
trained_classifier_weights = copy.deepcopy(net.classifier[0].weight.data)
init_classifier_weights = (1 + init_classifier_weights)*255
trained_classifier_weights = (1 + trained_classifier_weights)*255

init_classifier_weights = init_classifier_weights.view(-1,20,10).byte()
trained_classifier_weights = trained_classifier_weights.view(-1,20,10).byte()
if use_gpu:
    init_classifier_weights = init_classifier_weights.cpu()
    trained_classifier_weights = trained_classifier_weights.cpu()

d_weights = init_classifier_weights - trained_classifier_weights

fig = plt.figure()
plot=fig.add_subplot(1,3,1)
img = np.array(init_classifier_weights.numpy())[0]
plot.set_title('Initial Weights')
imgplot = plt.imshow(img)

plot=fig.add_subplot(1,3,2)
img = np.array(trained_classifier_weights.numpy())[0]
plot.set_title('Trained Weights')
imgplot = plt.imshow(img)

plot=fig.add_subplot(1,3,3)
img = np.array(d_weights.numpy())[0]
plot.set_title('Weight update')
imgplot = plt.imshow(img)
plt.show()